<h1>Emotion Recognition Inference with DeGirum PySDK</h1>

This notebook demonstrates emotion recognition using DeGirum PySDK on Hailo hardware.

In [ ]:
#!/usr/bin/env python3
import cv2
import numpy as np
import degirum as dg, degirum_tools

# Connection parameters
inference_host_address = "@local"
zoo_url = "degirum/hailo"
token = "<dg_token>"
# Device types can be adjusted as needed
device_type = "HAILORT/HAILO8L"

# Choose model names
face_det_model_name = "yolov8n_relu6_widerface_kpts--640x640_quant_hailort_hailo8l_1"
emotion_model_name = "emotion_recognition_fer2013--64x64_quant_hailort_multidevice_1"

# Choose image source
image_source = "<image_path>"

# Load the face detection model
print(f"Running face detection using '{face_det_model_name}' on image '{image_source}'")
face_model = dg.load_model(
    model_name=face_det_model_name,
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token=token,
    device_type=device_type
)

# Load the emotion classifier model
print(f"Loading emotion classifier '{emotion_model_name}'")
emotion_model = dg.load_model(
    model_name=emotion_model_name,
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token=token,
    device_type=device_type
)

# Perform face detection inference
face_detection_result = face_model(image_source)
print("Face Detection Results:", face_detection_result.results)

# Read the original image using OpenCV
image = cv2.imread(image_source)
if image is None:
    raise ValueError(f"Unable to read image from {image_source}")

# Process each detected face
if face_detection_result.results:
    for idx, detection in enumerate(face_detection_result.results):
        bbox = detection['bbox']  # Expected format: [x1, y1, x2, y2]
        x1, y1, x2, y2 = map(int, bbox)
        
        # Crop the face from the image
        face_crop = image[y1:y2, x1:x2]
        
        # Resize the face crop to 256x256 for emotion classification
        face_crop_resized = cv2.resize(face_crop, (256, 256))
        
        # Run emotion classification on the face crop
        emotion_result = emotion_model(face_crop_resized)
        final_emotion = max(emotion_result.results, key=lambda x: x['score'])
        print(f"Face {idx} final emotion: {final_emotion['label']}")
        
        # Annotate the face crop with the predicted emotion label
        cv2.putText(face_crop_resized, final_emotion['label'], (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # Display the annotated face crop using degirum_tools.Display
        with degirum_tools.Display(f"Face {idx} Emotion") as output_display:
            output_display.show_image(face_crop_resized)
else:
    print("No face detected.")